In [11]:
import math
from shapely.geometry import Point
import folium
import pandas as pd
import geopandas as gpd
import requests
import os
from dotenv import load_dotenv

load_dotenv(".env")
from tqdm import tqdm
import populartimes

In [12]:
def get_populartimes(restaurants_original):
    """
    Google Places APIの検索結果から混雑状況を取得する関数
    :param restaurants: Google Places APIの検索結果のGeoDataFrame(列:  name, place_id,geometry)
    
    """
    restaurants = restaurants_original.copy()
    API_KEY = os.getenv("API_KEY")

    # 初期化
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    # 各曜日の列を作成
    for day in weekdays:
        restaurants[day] = None 

    for idx, row in tqdm(restaurants.iterrows(), total=len(restaurants)):
        res = populartimes.get_id(API_KEY, row["place_id"])
        if res is not None:
            if "rating" in res:
                restaurants.loc[idx, "rating"] = res["rating"]
            if "rating_n" in res:
                restaurants.loc[idx, "rating_n"] = res["rating_n"]
            if "populartimes" in res:
                for day_data in res["populartimes"]:
                    day_name = day_data["name"]  # 'Monday' などの曜日名
                    if day_name in weekdays:
                        # 曜日ごとのデータをJSON形式で格納
                        restaurants.at[idx, day_name] = day_data["data"]

    return restaurants

In [13]:
# Google Places APIの検索結果を取得
restaurants = gpd.read_file("./data/results.geojson")
map = folium.Map(location=[35.681236, 139.767125], zoom_start=15)

# 検索結果から混雑状況を取得
populartimes = get_populartimes(restaurants)
populartimes

  0%|          | 0/193 [00:00<?, ?it/s]

100%|██████████| 193/193 [04:35<00:00,  1.43s/it]


,name,place_id,geometry,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,rating,rating_n
0,島食堂 ひなた,ChIJw5vHLO7vRDUR0oyp943HEho,POINT (131.71899 33.99085),None,None,None,None,None,None,None,4.1,24.0
1,ふぐ料理店専門店 栄ふく,ChIJ3ZqzmcfmRDURiZEaUKkaFI0,POINT (131.82837 34.02475),None,None,None,None,None,None,None,4.5,153.0
2,ラーメン並木 青果市場店,ChIJU2YJtr7mRDURsaWjpqYdOZQ,POINT (131.82497 34.02169),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 16, 29, 39, 29,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 21, 34, 40, 28...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 28, 53, 62, 38,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 28, 43, 43, 37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 74, 100, 97, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 45, 60, 75, 70, 60...",3.9,235.0
3,カラオケ コロッケ倶楽部 徳山店,ChIJz_ipCLbmRDURzcxYqIemoyU,POINT (131.83494 34.02048),None,None,None,None,None,None,None,3.5,127.0
4,ひろしま風お好み焼 ぷらむ,ChIJNX7SAcXmRDURiLPJ-R7CUpo,POINT (131.82748 34.03219),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33, 41, 37, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 37, 41, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 75, 95, 54, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 70, 54, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 87, 87, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 54, 62, 66, ...",4.1,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...
188,スナックコーナー 鹿野SA(上り),ChIJNf-sXfPaRDUReaKYwxgDZ5I,POINT (131.79527 34.22057),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 57, 76, 93, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33, 58, 61, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 41, 47, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 35, 25, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65, 69, 61, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 89, 88, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 88, 84, ...",3.6,117.0
189,木村屋,ChIJO7_ci2raRDURbH0rBAuxBE0,POINT (131.81798 34.21295),None,None,None,None,None,None,None,4.1,NaN
190,和寿,ChIJ59l3c0jbRDURxqo1YnC75qY,POINT (131.81342 34.22419),None,None,None,None,None,None,None,3.9,28.0
191,高美食堂,ChIJ7T6eanTaRDURQdEGQqzjk1w,POINT (131.81671 34.22748),None,None,None,None,None,None,None,3.8,NaN


In [19]:
def averaging_populartimes(populartimes_original,interval):
    """
    populartimesのデータを平均化する関数
    :param populartimes: populartimesのデータが格納されたGeoDataFrame
    :param interval: populartimesのデータの間隔(4時間なら4)
    :return: populartimesのデータを平均化したGeoDataFrame
    """
    populartimes = populartimes_original.copy()
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    averaged_weekdays = ["averaged_"+day for day in weekdays]
    populartimes[averaged_weekdays] = None

    for idx, row in populartimes.iterrows():
        for day in weekdays:
            if row[day] is not None:
                data = row[day]
                # 時間ごとのデータをintervalごとに平均化
                data = [sum(data[i:i+interval])/interval for i in range(0, len(data), interval)]
                populartimes.at[idx, "averaged_"+day] = data
        
        

    return populartimes

In [27]:
averaged_populartimes = averaging_populartimes(populartimes,4)


In [28]:
averaged_populartimes.to_file("./data/populartimes.geojson", driver="GeoJSON")